In [29]:
import cv2 , math
import numpy as np
import matplotlib.pyplot as plt
import os
from skimage.measure import label, regionprops, regionprops_table
import pandas as pd

from sklearn import svm, datasets
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder



    
# function to read the dir
def read_images_from_directory(root_directory):
    image_list = []

    for subdir, _, files in os.walk(root_directory):
        for file in files:
            file_path = os.path.join(subdir, file)
            
            # Check if the file is an image (you can add more image extensions if needed)
            if file_path.lower().endswith(('.bmp')):
                # Read the image
                img = cv2.imread(file_path)
                
                # Append the image and its file path to the list
                image_list.append((img, file_path))

    return image_list


    



directory_path_pre = ('/Users/zeyadhassan/Desktop/Digital image/12th /12th Project /PHASE I/output_pre')
images = read_images_from_directory(directory_path_pre)

directory_path_true = ('/Users/zeyadhassan/Desktop/Digital image/12th /12th Project /PHASE I/output')
true  = read_images_from_directory(directory_path_true)

    
    

#PHASE I
labelll = LabelEncoder()

features_tables = []
iou_val = []   
# for img , file_path in images:

for (img, file_path), (true_img, true_file_path) in zip(images, true):

        
    
    contrast = 6.5 # Contrast control ( 0 to 127)
    brightness = 1
    out = cv2.addWeighted( img, contrast, img, 0, brightness)
    
    
    median = cv2.medianBlur(out, 3)
    
    # Convert to HSV
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    
    # get the boundries for the purple color
    lower_bound = np.array([129,40,0])
    upper_bound = np.array([174, 255, 255])
    
    #mask of purple
    mask = cv2.inRange(hsv, lower_bound, upper_bound)
    
    # Extract the purple pixels using the mask
    purple = cv2.bitwise_and(img, img, mask=mask)
    # Remove noise
    purple = cv2.GaussianBlur(purple,(5,5),5)

    
    # Convert to grayscale and apply thresholding
    gray = cv2.cvtColor(purple, cv2.COLOR_BGR2GRAY)
    
    
    # Threshold the image to create a topographic map
    ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    #kernel for opening and closing 
    kernell = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7,7))
    
    #opening function
    open = cv2.morphologyEx(thresh, cv2.MORPH_OPEN,kernell,iterations=2)
    #closing function 
    close = cv2.morphologyEx(open, cv2.MORPH_CLOSE,kernell,iterations=3)
    
    
    #erosion
    kernel_size =[7,7]
    ellipse_kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, kernel_size)
    eroded = cv2.erode(close, ellipse_kernel, iterations = 3)
    #dilation
    kernel_size =[7,7]
    ellipse_kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, kernel_size) 
    dilated = cv2.dilate(eroded, ellipse_kernel, iterations =3)
    
    

    rgb = cv2.cvtColor(dilated, cv2.COLOR_BGR2RGB)
    
    
    imask = mask>0
    purple = np.zeros_like(img, np.uint8)
    purple[imask] = img[imask]
        
    
    
    

    # Convert the ground truth mask to grayscale
    truth_mask_gray = cv2.cvtColor(true_img, cv2.COLOR_BGR2GRAY)

    # Your existing IoU calculation code...
    rgb1 = cv2.cvtColor(rgb, cv2.COLOR_BGR2GRAY)
    # Calculate the intersection and union
    intersection = np.logical_and(rgb1, truth_mask_gray)
    union = np.logical_or(rgb1, truth_mask_gray)

    # Calculate the IoU score
    iou = np.sum(intersection) / np.sum(union)
    iou_val.append(iou)
    
    # plt.imshow(img)
    # plt.show()
    # plt.imshow(median)
    # plt.show()
    # plt.imshow(purple)
    # plt.show()
    
    # plt.imshow(gray)
    # plt.show()
    # plt.imshow(rgb)
    # plt.show()
    # plt.imshow(true_img)
    # plt.show()
    # contours, hierarchy = cv2.findContours(image=rgb, mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_NONE)
    # cv2.drawContours(image=rgb, contours=contours, contourIdx=-1, color=(255, 0, 0), thickness=1, lineType=cv2.LINE_AA)

# plt.plot(contours[1][:, 0][:, 0], contours[1][:, 0][:, 1], '-r', linewidth=2.5)
    # plt.imshow(rgb)
    # plt.show()
    

#PHASE II    
    
    
    
    bw  = rgb > 150
    label_img = label(bw)
    if len(label_img.shape) > 2:
        label_img = np.max(label_img, axis=2)
    regions = regionprops(label_img)
    props_dict = regionprops_table(label_img, rgb,
                               properties=[ 'euler_number','solidity' , 'perimeter' , 'eccentricity' ,'area'   ])
    features_table = [[] for label in props_dict['label']]

    features_table = pd.DataFrame(props_dict)  # Convert to DataFrame

    # print(props_dict)
    if "baso" in file_path:
        myfile_path = "baso"
    elif "eosi" in file_path:
        myfile_path = "eosi"
    elif "neut" in file_path:
        myfile_path = "neut"
    elif "lymp" in file_path:
        myfile_path = "lymp"
    elif "mixt" in file_path:
        myfile_path = "mixt"
    else:
        myfile_path = "mono"


    features_table['class'] = myfile_path

    features_tables.append(features_table)
    
all_features = pd.concat(features_tables, ignore_index=True)


# Display concatenated DataFrame


pd.set_option('display.max_rows', None)  # Show all rows without truncation
print(all_features)   


mybaso = all_features[all_features['class'] == 'baso']
mymono = all_features[all_features['class'] == 'mono']
mylymp = all_features[all_features['class'] == 'lymp']
myneut = all_features[all_features['class'] == 'neut']
myeosi = all_features[all_features['class'] == 'eosi']
mymixt = all_features[all_features['class'] == 'mixt']
class_arrays = [mybaso, mymono, mylymp, myneut, myeosi, mymixt]


# for class_array in class_arrays:
# for index, class_array in enumerate(class_arrays):
#     class_name = class_array.iloc[0]['class']
#     X = class_array.iloc[:, 0:5]  # Assuming columns 2 to 6 are features, adjust as needed
#     y = labelll.fit_transform(features_table['class'])
# # X = mybaso.iloc[:, 2:7]  # Assuming columns 2 to 6 are features, adjust as needed
# # y = mybaso.iloc[:, 0]   # Assuming the first column is the label

# # Plotting the scatter plot for the first two columns of X
#     # plt.figure(figsize=(12, 12))
#     # plt.scatter(X.iloc[:, 2:3], X.iloc[:, 4:5], c=y, cmap=plt.cm.Set1, edgecolor="k")
#     # plt.xlabel('Feature 1')  # Replace with meaningful feature name if available
#     # plt.ylabel('Feature 2')  # Replace with meaningful feature name if available
#     # plt.title('Scatter plot: Feature 1 vs Feature 2')
#     # plt.colorbar(label='Labels')
#     # plt.show()
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
#     clf = svm.SVC(kernel="linear", C = 1)
#     clf.fit(X_train, y_train)
#     clf_predictions = clf.predict(X_test)

#     accuracy = clf.score(X_test, y_test) * 100
    
#     # print("Accuracy:  ",accuracy)
#     print(f"Class Array {index}: {class_name} - Accuracy: {accuracy}")


X = all_features.iloc[:, 0:5]  
y = labelll.fit_transform(all_features['class']) 
# plt.figure(figsize=(12, 12))
# plt.scatter(X.iloc[:, 2:3], X.iloc[:, 4:5], c=y, cmap=plt.cm.Set1, edgecolor="k")
# plt.xlabel('Feature 1')  # Replace with meaningful feature name if available
# plt.ylabel('Feature 2')  # Replace with meaningful feature name if available
# plt.title('Scatter plot: Feature 1 vs Feature 2')
# plt.colorbar(label='Labels')
# plt.show()
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
# clf = svm.SVC(kernel="linear", C = 1)
# clf.fit(X_train, y_train)
# clf_predictions = clf.predict(X_test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)
clf = svm.SVC(kernel="linear", C = 1)
clf.fit(X_train, y_train)
clf_predictions = clf.predict(X_test)
accuracy = clf.score(X_test, y_test) * 100

print("Total Accuracy: ",accuracy)

 
avg = np.mean(iou_val) * 100
print("Average IoU:",avg)    
    
    
  
    
    






     label  euler_number  solidity  eccentricity     area class
0        1             1  0.971765      0.528748   1239.0  lymp
1        1             1  0.966871      0.560404   2685.0  lymp
2        1             1  0.977690      0.508678    745.0  lymp
3        2             1  0.956002      0.713176   2238.0  lymp
4        1             1  0.968898      0.608776   2461.0  lymp
5        1             1  0.960181      0.755531   1905.0  lymp
6        1             1  0.975788      0.665130   2539.0  lymp
7        1             1  0.971317      0.336124   1490.0  lymp
8        1             1  0.965388      0.597462   1283.0  lymp
9        1             1  0.965857      0.539395   1471.0  lymp
10       1             1  0.968736      0.668233   2231.0  lymp
11       1             1  0.964474      0.352237   2199.0  lymp
12       2             1  0.957688      0.600815   1856.0  lymp
13       1             1  0.961062      0.625092    543.0  lymp
14       2             1  0.970560      